In [12]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from features.samplers import PairSampler, TripletSampler
from features.settings import Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from keras.datasets import mnist, fashion_mnist, cifar10, cifar100
from sklearn.svm import LinearSVC
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def test_model(model, settings, data_sampler, data, parameters, num_steps):
    
    num_features, lr, reg, drop = parameters
    (train_x, train_y), (test_x, test_y) = data
    train_sampler, test_sampler = data_sampler(train_x, train_y), data_sampler(test_x, test_y)
    
    x = tf.placeholder(tf.float32, shape=[None, 28, 28])
    y = tf.placeholder(tf.int64, shape=[None])
    p = tf.placeholder(tf.float32)
    model = model(x, y, p, settings, parameters)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training // parameters:", parameters)
        train_time = time.time()
        for step in range(num_steps):
            x_, y_ = train_sampler.sample(100)
            sess.run(model.optimize, feed_dict={x:x_, y:y_, p:drop})
            
            if step % 5 == 0:                
                train_loss, train_acc = sess.run(model.metrics, feed_dict={x:x_, y:y_, p:1})                
                x_, y_ = test_sampler.sample(1000)
                test_loss, test_acc = sess.run(model.metrics, feed_dict={x:x_, y:y_, p:1})
                print("\tstep %d: train loss %g, train acc %g, test loss %g, test acc %g"%
                      (step, train_loss, train_acc, test_loss, test_acc))  
                
                svc = LinearSVC(random_state=0)
                train_features = sess.run(model.features, feed_dict={x:train_x, y:train_y, p:1}) 
                svc.fit(train_features, train_y)
                test_features = sess.run(model.features, feed_dict={x:test_x, y:test_y, p:1})
                print("\tclassification accuracy: {:.4f}".format(svc.score(test_features, test_y)))
        train_time = time.time() - train_time
        print("end training // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = mnist.test.next_batch(1000)
        test_acc = sess.run(model.acc, feed_dict={x:x_, y:y_, p:1})
        eval_test_time = time.time() - eval_test_time
        print("test set acc: %.4f // time elapsed: %.4f s"%(test_acc, eval_test_time))
        
        svc = LinearSVC(random_state=0)
        train_features = sess.run(model.features, feed_dict={x:train_x, y:train_y, p:1}) 
        svc.fit(train_features, train_y)
        test_features = sess.run(model.features, feed_dict={x:test_x, y:test_y, p:1})
        print("\tclassification accuracy: {:.4f}".format(svc.score(test_features, test_y)))

In [22]:
# (train_x, train_y), (test_x, test_y) = mnist.load_data()
# N_train, _, _ = train_x.shape
# N_test, _, _ = test_x.shape

In [23]:
# data = (train_x.reshape((N_train,-1)), train_y), (test_x.reshape((N_test,-1)), test_y)

In [24]:
parameters = (10, 0.0001, 0, 0.8)
test_model(ConvModelSmall, Pair_Hinge_Settings, PairSampler, mnist.load_data(), parameters, 10)

begin training // parameters: (10, 0.0001, 0, 0.8)
	step 0: train loss 531.114, train acc 0.0996667, test loss 543.998, test acc 0.1022
	classification accuracy: 0.4463
	step 5: train loss 204.669, train acc 0.101916, test loss 210.792, test acc 0.0993333


KeyboardInterrupt: 